# Pubs in San Mateo.

## 1º Import the libraries 

In [1]:
import requests
import json
from dotenv import load_dotenv
import os
import sys
import pandas as pd
from functools import reduce
import operator
import geopandas as gpd
from pymongo import MongoClient,GEOSPHERE
import shapely.geometry
import funtions as fc

## 2º Query.

In [2]:
df_query_san_mateo = fc.query("pubs",5,1000)

In [3]:
gdf_query_san_mateo = fc.geo_data(df_query_san_mateo)

In [4]:
fc.point_geometry(gdf_query_san_mateo)

/usr/local/lib/python3.9/site-packages/geopandas/geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


,name,latitud,longitud,geometry
0,O'Neill's Irish Pub,37.567145,-122.324384,"{'type': 'Point', 'coordinates': (-122.3243842..."
1,Moon's Family Sports Pub,37.566127,-122.322821,"{'type': 'Point', 'coordinates': (-122.3228214..."
2,3rd Avenue Sports Bar & Grill,37.564051,-122.324639,"{'type': 'Point', 'coordinates': (-122.3246390..."
3,O'Niel's,37.567162,-122.323764,"{'type': 'Point', 'coordinates': (-122.3237638..."


## 3º Import the query to Mongo in a new collection.

In [5]:
client = MongoClient() 
db = client.ironhack
collection = db.pubs 

In [6]:
collection.create_index ([("geometry", GEOSPHERE)])
data = gdf_query_san_mateo.to_dict(orient='records')
collection.insert_many(data)

## 4º Select the nearest pubs.

In [7]:
db.pubs 

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack'), 'pubs')

In [8]:
filt = {"geometry":{"$near":{"type":"Point","coordinates":[-122.323895,37.566879]}}}
proyect = {"_id":0,"name":1,"latitud":1,"longitud":1}

result = db.pubs.find(filt,proyect).limit(1)
pub = list(result)

In [9]:
pub

[{'name': "O'Niel's",
  'latitud': 37.56716214181553,
  'longitud': -122.32376388773964}]

## 5º Map with all pubs. 

In [10]:
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd

In [11]:
inicial_lat = 37.566879
inicial_long = -122.323895

In [12]:
maps = folium.Map(location = [inicial_lat,inicial_long], zoom_start=15)

In [13]:
San_Mateo = Marker(location = [inicial_lat,inicial_long],tooltip="New office")
San_Mateo.add_to(maps)

In [19]:
for i,row in df_query_san_mateo.iterrows():
    pubs = {
        "location" : [row["latitud"],row["longitud"]],
        "tooltip" : row["name"]    
    }
    if row["latitud"] == 37.56716214181553 and row["longitud"]== -122.32376388773964 :
        icon = Icon(color = "green",
                    prefix = "fa",
                    icon = "glass",
                    icon_color = "black"
        )
    else:
        icon = Icon(color = "red",
                    prefix = "fa",
                    icon = "glass",
                    icon_color = "black"
        )
 
    Marker(**pubs,icon = icon).add_to(maps)
maps
